Historical Tweet API

In [1]:
%config IPCompleter.greedy=True

In [2]:
from datetime import datetime as dt
from datetime import timedelta as td
import requests
import pandas as pd
import base64
import csv

In [3]:
client_key = ''
client_secret = ''
BEARER_TOKEN = ""

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [4]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query':  'covid OR bitcoin OR cryptocurrency',
    'max_results': '100',
    'tweet.fields': 'created_at,lang'
}

#'bitcoin OR ETH OR LINK OR bitcoin covid OR XRP'

In [5]:
#Function to create dataframe of interest
def get_data(tweet):
    data = {
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'text': tweet['text'],
        'lang': tweet['lang']
        
    }
    return data

In [6]:
# Gather the last 7 days of tweets

dtformat = '%Y-%m-%dT%H:%M:%SZ'  # the date format string required by twitter

# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    now = dt.strptime(now, dtformat)
    back_in_time = now - td(minutes=mins)
    return back_in_time.strftime(dtformat)
    
now = dt.now()  # get the current datetime, this is our starting point
last_week = now - td(days=7)  # datetime one week ago = the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API
df = pd.DataFrame()  # initialize dataframe to store tweets
while True:
    if dt.strptime(now, dtformat) < last_week:
        # if we have reached 7 days ago, break the loop
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    params['start_time'] = pre60
    params['end_time'] = now
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
    
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    if 'data' in response.json():
        for tweet in response.json()['data']:
            row = get_data(tweet)  # we defined this function earlier
            df = df.append(row, ignore_index=True)

In [7]:
# Expand columns to read entire tweet
pd.set_option('max_colwidth', None)

In [8]:
df

,created_at,id,lang,text
0,2021-01-14T20:15:39.000Z,1349812445620015106,en,"RT @Oahu_DEM: Did you know that a CDC model found that 59% of COVID-19 transmission came from people WITHOUT symptoms, either from before t…"
1,2021-01-14T20:15:39.000Z,1349812445569835008,pt,@ZeSimoes1 @NCarvalho_Br @CarlaZambelli38 @fiocruz @govbr Vou postar aqui... Para reforçar minha opinião.\n\nhttps://t.co/f3XCPCsf8e
2,2021-01-14T20:15:39.000Z,1349812445569802240,es,"RT @CZGNBLaGuaira: Buenos días pueblo aragueño, la .@GNB_Aragua se mantiene desplegada velando por la seguridad de la población y recordand…"
3,2021-01-14T20:15:39.000Z,1349812445540442112,en,"RT @NeilDotObrien: As the UK tragically hits a record number of Covid-19 deaths, Covid-sceptic-in-chief @toadmeister appears to have delete…"
4,2021-01-14T20:15:39.000Z,1349812445536202768,en,"RT @LegionHoops: Nets GM Sean Marks says that Kyrie Irving is “excited” to return, and has been testing for COVID-19 daily."
...,...,...,...,...
16008,2021-01-08T03:15:35.000Z,1347381409355231234,en,"@joebob2020bb If it wasn’t COVID, Trump would have won."
16009,2021-01-08T03:15:35.000Z,1347381409271468035,en,"RT @SethAbramson: 4,134 dead of COVID-19 today. The worst day of the pandemic. The second-worst day was yesterday (4,100). Incredible to th…"
16010,2021-01-08T03:15:35.000Z,1347381408868786176,en,RT @BURNERBBABBY: Pre covid New York was a movie https://t.co/PtcaePsBvA
16011,2021-01-08T03:15:35.000Z,1347381408839442438,es,RT @RiveraNat: Mi mamá y yo dimos positivo para covid-19 han sido los días más difíciles de mi vida. Tener que batallar contra este monstru…


In [9]:
#Filter out any rows that are not english
data = df[df["lang"] == 'en']

In [10]:
data

,created_at,id,lang,text
0,2021-01-14T20:15:39.000Z,1349812445620015106,en,"RT @Oahu_DEM: Did you know that a CDC model found that 59% of COVID-19 transmission came from people WITHOUT symptoms, either from before t…"
3,2021-01-14T20:15:39.000Z,1349812445540442112,en,"RT @NeilDotObrien: As the UK tragically hits a record number of Covid-19 deaths, Covid-sceptic-in-chief @toadmeister appears to have delete…"
4,2021-01-14T20:15:39.000Z,1349812445536202768,en,"RT @LegionHoops: Nets GM Sean Marks says that Kyrie Irving is “excited” to return, and has been testing for COVID-19 daily."
6,2021-01-14T20:15:39.000Z,1349812444806471682,en,"RT @PHE_uk: Due to a processing issue with deaths data the #COVID19 Dashboard update is delayed today.\n\n48,682 new COVID-19 positive cases…"
9,2021-01-14T20:15:39.000Z,1349812444340887555,en,Record daily #German #COVID-19 deaths spark #Merkel 'mega-lockdown' plan: Bild https://t.co/GSCjjYcW9c #COVID19 #CovidVaccine #coronavirus
...,...,...,...,...
16006,2021-01-08T03:15:35.000Z,1347381409577455616,en,"@NeveltenJohn @greggnunziata At every turn when Trump could have abused his power he didn’t. He words were harsh, loud, mean and sometimes hurtful. His actions were not. He could have use this Covid thing to grab power like Democrats did. Instead he resisted. You have been deceived by media."
16007,2021-01-08T03:15:35.000Z,1347381409384718337,en,RT @barstoolsports: The Covid Stimulus Bill Requires the Pentagon to Tell Us What it Knows About UFOs https://t.co/m78iCRxqj0 https://t.co/…
16008,2021-01-08T03:15:35.000Z,1347381409355231234,en,"@joebob2020bb If it wasn’t COVID, Trump would have won."
16009,2021-01-08T03:15:35.000Z,1347381409271468035,en,"RT @SethAbramson: 4,134 dead of COVID-19 today. The worst day of the pandemic. The second-worst day was yesterday (4,100). Incredible to th…"


In [11]:
data.to_csv('C:\\Users\jwkon\Data Learning\Sentiment Analysis\covidORbitcoinORcryptocurrency.csv')